In [ ]:
import dataset, utils, net, loss

import torch, random, torchvision, importlib
from PIL import ImageDraw
from IPython.display import display

torch.set_printoptions(linewidth=240, threshold=10000, sci_mode=False, precision=3)


Load Dataset

In [ ]:
importlib.reload(dataset)
importlib.reload(utils)

cv = utils.Converter()
rd = dataset.RisikoDataset(dataset_dir='Datasets/Generated_Dataset/val', cv=cv, train_mode=True)

Test Dataset and Conversion of labels

In [ ]:
colors = ( "blue", "red", "yellow", "purple", "black", "green")

rnd_idx = random.randint(a=0, b=rd.__len__()-1)
# rnd_idx = 326
img_tensor, netout_labels = rd[rnd_idx]

boxes = rd.cv.convert_labels_to_basic_labels(netout_labels)
# rd.cv.convert_labels_from_relative_to_absolute_values(boxes, use_netin_shape=True)

print(netout_labels[netout_labels[...,0] == 1])
print(boxes)

to_pil = torchvision.transforms.ToPILImage()
img = to_pil(img_tensor)

if (boxes.shape[0] != 0):

    if len(boxes.shape) == 1:
        boxes = boxes.unsqueeze(1)

    box_coords = boxes[:, 2:]
    torch.multiply(box_coords, torch.tensor([img.width, img.height, img.width, img.height], dtype=torch.float32), out=box_coords)
    print(box_coords)

    img_draw = ImageDraw.Draw(img)

    for i in range(box_coords.shape[0]):

        x0 = int(box_coords[i,0] - box_coords[i,2] / 2)
        x1 = int(box_coords[i,0] + box_coords[i,2] / 2)
        y0 = int(box_coords[i,1] - box_coords[i,3] / 2)
        y1 = int(box_coords[i,1] + box_coords[i,3] / 2)

        img_draw.rectangle([x0, y0, x1, y1], outline=colors[int(boxes[i,1])%6])

display(img)

Network forward test

In [ ]:
importlib.reload(net)

my_net = net.DetektorNet(abox_count=cv.abox_count)

rnd_idx = random.randint(a=0, b=rd.__len__()-1)
# rnd_idx = 313
print('Chosen index = ' + str(rnd_idx))
img_tensor, netout_labels = rd[rnd_idx]
print(netout_labels.shape)
img_tensor = img_tensor.type(cv.netout_dtype).unsqueeze(0)

my_net.train()
netout = my_net.forward(img_tensor)
print('Train shape: ' + str(netout.shape))
my_net.eval()
netout = my_net.forward(img_tensor)
print('Eval shape: ' + str(netout.shape))

Loss test

In [ ]:
importlib.reload(loss)

my_loss = loss.RisikoLoss(
    lambda_abox=0.1,
    lambda_coord=1,
    lambda_no_obj=0.05,
    lambda_class_obj=1,
    lambda_class_color=0.1,
    abox_count=cv.abox_count
)

artificial_labels = torch.zeros(size=(3,10,8), dtype=cv.netout_dtype)
artificial_labels[0,1] = torch.tensor((1,1,2,0.1,0.1,0,0.3,-0.2))
artificial_labels[0,2] = torch.tensor((1,1,5,0.1,0.1,1,0.3,-0.2))
artificial_labels[1,0] = torch.tensor((1,1,3,0.1,0.1,0,0.3,-0.2))

artificial_netout = torch.zeros(size=(3,10,19), dtype=cv.netout_dtype)
artificial_netout[0,1] = torch.tensor((1,1,0,0,10,0,0,0,0.1,0.1,10,0,0,0.3,-0.2,0,0,0,0))
artificial_netout[0,2] = torch.tensor((1,1,0,0,0,0,0,10,0.1,0.1,0,10,0,0,0,0.3,-0.2,0,0))
artificial_netout[1,0] = torch.tensor((1,1,0,0,0,10,0,0,0.1,0.1,10,0,0,0.3,-0.2,0,0,0,0))

print(my_loss(artificial_netout, artificial_labels))
